In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string
import re
import nltk
nltk.download('punkt')

# Load the dataset
train_df = pd.read_csv('train.csv')
qa_df = pd.read_csv('QA_data.csv', encoding='ISO-8859-1')

# Define a function to normalize and tokenize the text
def normalize_and_tokenize(text):
    # Remove special characters and punctuation symbols
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    return tokens

# Apply the function to the 'text' column
train_df['text'] = train_df['text'].apply(normalize_and_tokenize)
qa_df['text'] = qa_df['text'].apply(normalize_and_tokenize)
qa_df['answer'] = qa_df['answer'].apply(normalize_and_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip glove.6B.300d.txt

--2023-05-19 17:54:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-19 17:54:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-19 17:54:36--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import numpy as np

# Load the pre-trained GloVe embeddings
embeddings_index = {}
with open('glove.6B.300d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Define a function to get the embedding for a given word
def get_embedding(word):
    embedding = embeddings_index.get(word)
    if embedding is not None:
        return embedding
    else:
        return np.zeros(300)

# Define a function to get the embeddings for a given text
def get_text_embeddings(text):
    embeddings = []
    for word in text:
        embedding = get_embedding(word)
        embeddings.append(embedding)
    return embeddings

# Apply the function to the 'text' column of the train and test datasets
train_df['text_embeddings'] = train_df['text'].apply(get_text_embeddings)
qa_df['text_embeddings'] = qa_df['text'].apply(get_text_embeddings)
qa_df['answer_embeddings'] = qa_df['answer'].apply(get_text_embeddings)

In [ ]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Dense
from keras.utils import to_categorical
!pip install keras_preprocessing
from keras_preprocessing.sequence import pad_sequences

# Set the number of classes
num_coarse_classes = 6
num_fine_classes = 50

# Convert the 'label-coarse' and 'label-fine' columns to one-hot encoded labels
y_coarse = to_categorical(qa_df['label-coarse'], num_classes=num_coarse_classes)
y_fine = to_categorical(qa_df['label-fine'], num_classes=num_fine_classes)

# Define the maximum sequence length
max_seq_length = 100

# Add a <PAD> token at the end of each text sequence
qa_df['padded_text'] = qa_df['text'].apply(lambda x: x + ['<PAD>'])

# Get the embeddings for the padded text sequences
X_text = qa_df['padded_text'].apply(get_text_embeddings)

# Pad or truncate the 'text_embeddings' sequences to have the same length
X_text = pad_sequences(X_text, maxlen=max_seq_length + 1, dtype='float32')

# Define the input shape
input_shape = (max_seq_length + 1, 300)

# Create the input layer
inputs = Input(shape=input_shape)

# Create the Bidirectional LSTM layer
lstm_outputs = Bidirectional(LSTM(100, return_sequences=True))(inputs)

# Create the coarse and fine output layers
coarse_outputs = Dense(num_coarse_classes, activation='softmax')(lstm_outputs[:, -2, :])
fine_outputs = Dense(num_fine_classes, activation='softmax')(lstm_outputs[:, -1, :])

# Create the model
model = Model(inputs=inputs, outputs=[coarse_outputs, fine_outputs])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_text, [y_coarse, y_fine], epochs=50, batch_size=32)

questions = ['How many people speak French?', 'What day is today?', 'Who will win the war?', 'Who is Italian first minister?', 'When World War II ended?', 'When Gandhi was assassinated?']

for question in questions:
    # Normalize and tokenize the question
    tokens = normalize_and_tokenize(question)
    # Add a <PAD> token at the end of the sequence
    tokens.append('<PAD>')
    # Get the embeddings for the tokens
    embeddings = get_text_embeddings(tokens)
    # Pad or truncate the sequence to have the same length as the input shape
    embeddings = pad_sequences([embeddings], maxlen=max_seq_length + 1, dtype='float32')
    # Predict the coarse and fine labels for the question
    coarse_pred, fine_pred = model.predict(embeddings)
    # Get the index of the predicted coarse and fine labels
    coarse_index = np.argmax(coarse_pred)
    fine_index = np.argmax(fine_pred)
    # Find all rows in qa_df where 'label-coarse' == coarse_index and 'label-fine' == fine_index
    rows = qa_df[(qa_df['label-coarse'] == coarse_index) & (qa_df['label-fine'] == fine_index)]
    if len(rows) > 0:
        # Select a random row from rows as an answer to this question.
        answer_row = rows.sample(n=1)
        answer_text = answer_row.iloc[0]['answer']
        print(f"Q: {question}")
        print(f"A: {' '.join(answer_text)}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00
Epoch 1/50
16/16 [==============================] - 10s 271ms/step - loss: 5.2050 - dense_loss: 1.5668 - dense_1_loss: 3.6382 - dense_accuracy: 0.4220 - dense_1_accuracy: 0.2700
Epoch 2/50
16/16 [==============================] - 4s 261ms/step - loss: 3.8524 - dense_loss: 1.1233 - dense_1_loss: 2.7291 - dense_accuracy: 0.6520 - dense_1_accuracy: 0.4180
Epoch 3/50
16/16 [==============================] - 6s 359ms/step - loss: 3.0258 - dense_loss: 0.8122 - dense_1_loss: 2.2136 - dense_accuracy: 0.7760 - dense_1_accuracy: 0.5080
Epoch 4/50
16/16 [==============================] - 4s 261ms/step - loss: 2.5422 - dense_loss: 0.6074 - dense_1_loss: 1.9348 - dense_accuracy: 0.8200 - dense_1_accuracy: 0.5280
Epoch 5/50
16/16 [==============================] - 5s 305ms/step - loss: 2.0974 - dense_loss: 0.4098 - dense_1_

In [ ]:
questions = ['How many people speak French?', 'What day is today?', 'Who will win the war?', 'Who is Italian first minister?', 'When World War II ended?', 'When Gandhi was assassinated?']

for question in questions:
    # Normalize and tokenize the question
    tokens = normalize_and_tokenize(question)
    # Add a <PAD> token at the end of the sequence
    tokens.append('<PAD>')
    # Get the embeddings for the tokens
    embeddings = get_text_embeddings(tokens)
    # Pad or truncate the sequence to have the same length as the input shape
    embeddings = pad_sequences([embeddings], maxlen=max_seq_length + 1, dtype='float32')
    # Predict the coarse and fine labels for the question
    coarse_pred, fine_pred = model.predict(embeddings)
    # Get the index of the predicted coarse and fine labels
    coarse_index = np.argmax(coarse_pred)
    fine_index = np.argmax(fine_pred)
    # Find all rows in qa_df where 'label-coarse' == coarse_index and 'label-fine' == fine_index
    rows = qa_df[(qa_df['label-coarse'] == coarse_index) & (qa_df['label-fine'] == fine_index)]
    if len(rows) > 0:
        # Select a random row from rows as an answer to this question.
        answer_row = rows.sample(n=1)
        answer_text = answer_row.iloc[0]['answer']
        print(f"Q: {question}")
        print(f"A: {' '.join(answer_text)}")

1/1 [==============================] - 0s 80ms/step
Q: What day is today?
A: measures temperature
1/1 [==============================] - 0s 132ms/step
Q: Who will win the war?
A: chris haney and scott abbott
1/1 [==============================] - 0s 161ms/step
Q: Who is Italian first minister?
A: the bard
1/1 [==============================] - 0s 231ms/step
Q: When World War II ended?
A: 1789
1/1 [==============================] - 0s 178ms/step
Q: When Gandhi was assassinated?
A: 1939
